In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import requests
import json
import time

from random import randint
from math import exp

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
#Read in democrat and republican baseline data
df_dems_1 = pd.read_csv('./Data/InitialDemocratsPosts_8_31.csv')
df_dems_2 = pd.read_csv('./Data/InitialDemocratsPosts_9_03.csv')

'''One post (stickied at the top) was locked and showed up in both hits on r/dem. 
Removed it since our model is about number of posts, not strength or relevancy of those posts.'''
df_democrats_all = df_dems_1.append(df_dems_2.iloc[1:, ], ignore_index=True)

df_reps_1 = pd.read_csv('./Data/InitialRepublicanPosts.csv')
df_reps_2 = pd.read_csv('./Data/InitialRepublicanPosts_9_03.csv')
df_republicans_all = df_reps_1.append(df_reps_2, ignore_index=True)

#Null group sets
df_all_init = pd.read_csv('./Data/allPosts_9_04_A.csv')
df_cats_init = pd.read_csv('./Data/catsPosts_9_03_A.csv')
df_dogs_init = pd.read_csv('./Data/dogsPosts_9_03_A.csv')
df_hbb_init = pd.read_csv('./Data/hbbPosts_9_04_A.csv')

#Join all to single dataframe
df_init_DemRep = df_democrats_all.append(df_republicans_all, ignore_index=True, verify_integrity=True)
for df in [df_all_init,df_cats_init, df_dogs_init, df_hbb_init]:
    df_init_DemRep =  df_init_DemRep.append(df, ignore_index=True, verify_integrity=True)
    
#confirm 4004 posts included
print('    Data Shape: ',df_init_DemRep.shape) 
print('    Duplicate headers (expect count 2): ', len([label for label in df_init_DemRep['subreddit'].value_counts().index if label =='democrats' or label =='Republican']))  #Expect 2

#Translate to 0, 1, 2 groups
df_init_DemRep['subred_cat'] = (df_init_DemRep['subreddit']=='democrats')*2 + (df_init_DemRep['subreddit']=='Republican')*1
df_init_DemRep.fillna(' ', inplace=True)
print('    nulls: ', df_init_DemRep.isnull().sum())
print('    new counts: ', df_init_DemRep['subred_cat'].value_counts())

    Data Shape:  (4004, 3)
    Duplicate headers (expect count 2):  2
    nulls:  selftext      0
subreddit     0
title         0
subred_cat    0
dtype: int64
    new counts:  0    2003
2    1001
1    1000
Name: subred_cat, dtype: int64


In [49]:
def subreddit_getter(subreddit_source, your_name_bot, max_pulls=20):
    """Subreddit source is the sub to be pulled from. should be literal string. E.G., /r/football should be input as 'football'
    your_name_bot will be used to associate calls on your machine.
    max_pulls is number of times to query subreddit data
    
    The resulting dataframe from this process is available globally as df_latest_scrape after completion. 
    
    NB: Save function assumes Data subfolder exists in working directory.
    """
    data_list = []
    base_url = "https://www.reddit.com/r/" + subreddit_source + "/.json"
    URL = base_url

    #seems to be a cap around 20 for some smaller subs
    #cur_name = data['data']['children'][i]['data']['subreddit_name_prefixed'].replace('r/', '')
    for _ in range(max_pulls):
        try:
            data = requests.get(URL, headers={'User-agent': your_name_bot}).json()
            
            for i in range(len(data['data']['children'])):
                temp = {}
                temp['subreddit'] = subreddit_source
                temp['title']=data['data']['children'][i]['data']['title']
                data_list.append(temp)
            
            new_after = data['data']['after']
            URL = base_url + "?after=" + new_after
                
            time.sleep(2)  #very important to avoid getting locked out by Reddit.
        except: 
            break
    #Create dataframe, write to csv, save to notebook
    df_scrape = pd.DataFrame(data_list)
    random_name = subreddit_source + str(randint(1, 100000)) #Randint used to avoid writing over previous csvs
    df_scrape.to_csv('./Data/' + random_name + '.csv')
    
    global df_latest_scrape
    df_latest_scrape = df_scrape

In [3]:
#Below is the fit_transform function

In [4]:
my_stop_words = stop_words.ENGLISH_STOP_WORDS
#no compelling reason to pull in self-text
df_data = df_init_DemRep.loc[:, ['subred_cat', 'title']]
def fit_transform_model(X=df_data.title, y=df_data.subred_cat, 
                        vecmod='TFID', pred='logreg', 
                        test_size_local=None, random_state_local=42 ):
    """
    X data pre-transform, y data classes.
    vecmod - some vectorizer. Default is TF-ID. Must put fully instatiated vectorizer in here to update
    pred - predictor. default is logisticregression
    test_size_local - split size for train test split
    random_state for train test split
    """
    
    #get vectorizer
    if vecmod == 'TFID':
        vecmod_ = TfidfVectorizer(stop_words=my_stop_words, ngram_range=(1,3))
    else:
        vecmod_ = vecmod
    
    #TTS, fit transform on x_train, convert back to pd.DataFrame for later feature extraction
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size_local, 
                                                        random_state=random_state_local)
    X_train_transform = vecmod_.fit_transform(X_train).todense()
    df_X_train_transform = pd.DataFrame(X_train_transform, columns=vecmod_.get_feature_names())
    X_test_transform = vecmod_.transform(X_test).todense()
    df_X_test_transform = pd.DataFrame(X_test_transform, columns=vecmod_.get_feature_names())
    
    #predict model
    if pred == 'logreg':
        funct_model_= LogisticRegression()
    else:
        funct_model_ = pred
        
    funct_model_.fit(df_X_train_transform, y_train)
    
    #Export features and scores
    global latest_model, latest_transform, sorted_d_values_none, sorted_d_values_rep, sorted_d_values_dem, train_x, test_x, test_y, train_y
    latest_model = funct_model_ 
    train_x, test_x, train_y, test_y = df_X_train_transform, df_X_test_transform, y_train, y_test
    latest_transform = vecmod_
    
    try:
        #create feature dictionaries of each word for each category. only sorted_d is exported 
        feature_values_none =(dict(zip(list(df_X_train_transform.columns), list(funct_model_.coef_[0]))))
        sorted_d_values_none = sorted({(value, key) for (key,value) in feature_values_none.items()}, reverse=True)
        
        feature_values_rep =(dict(zip(list(df_X_train_transform.columns), list(funct_model_.coef_[1]))))
        sorted_d_values_rep = sorted({(value, key) for (key,value) in feature_values_rep.items()}, reverse=True)
        
        feature_values_dem =(dict(zip(list(df_X_train_transform.columns), list(funct_model_.coef_[2]))))
        sorted_d_values_dem = sorted({(value, key) for (key,value) in feature_values_dem.items()}, reverse=True)

    except:
        print('   broken sorter')
        return funct_model_.score(df_X_test_transform, y_test)
    
    return funct_model_.score(df_X_test_transform, y_test)

In [106]:
fit_transform_model(test_size_local=.2, vecmod=CountVectorizer())

0.9038701622971286

In [1]:
def subreddit_Analysis(df_data_current_, output_figname=None, bump_dems=True):
    """
    df_data_current is an untransformed dataframe of post titles, with 2 columns: subreddit, and title
    output_figname: if none (default), do not output and save score distribution. otherwise, use name provided. 
    output_figname assumes that there is a subfolder in current working directory:  ./Presentation/Outputfigs/
    bump_dems: if True (default), include systematic adjustment for democratic posts
    """
    #transform list of posts to dataframe for analysis
    X_cur = latest_transform.transform(df_data_current_.title).todense()
    df_cur_predictions = pd.DataFrame(latest_model.predict(X_cur), columns=['Party'])
    df_cur_predictions = df_cur_predictions.join(pd.DataFrame(latest_model.predict_proba(X_cur), columns = ['none_score','rep_score', 'dem_score'] ))
    
    
    '''
    Systematic adjustment for built in model bias. Necessary for intercept tuning in the wild. 
    See discussion: How strong are the resulting proba scores for each post? below
    Arrived at .45 adjustment after calibration. Otherwise, simulates setting intercepts equal to 
    eachother in predicted probas calculation.
    '''
    if bump_dems:
        dem_bump = abs(exp(latest_model.intercept_[2])-exp(latest_model.intercept_[1]))*.45
    else:
        dem_bump = 0
    df_cur_predictions['dem_score'] = df_cur_predictions['dem_score'] + dem_bump
    
    
    #In this section, we force the model to pick dem or rep for every post. Then we rescore it, and output a nice figure that visualizes the spread 
    '''The next 3 rows mimic the output of a 2 category model with type republican =0, democrat=1. 
    Since we are asking, what is the dominant party ranking for this sub, other than none, we want some intentionally biased results.'''
    df_cur_predictions['NewDem'] = 1*(df_cur_predictions['dem_score'] > df_cur_predictions['rep_score']) #Score whichever is higher proba
    #create 'Normalized' probabs based on the assumption that a post is either republican or democrat and never neither 
    df_cur_predictions['NewDem_proba'] = df_cur_predictions['dem_score']/(df_cur_predictions['dem_score']+df_cur_predictions['rep_score'])
    df_cur_predictions['NewRep_proba'] = 1 - df_cur_predictions['NewDem_proba']


    """
    Return: 
    subreddit: name of subreddit
    Init_intensity: number of posts identified as Democratic or Republican before coersion. Used as a proxy for baseline level of political-ness of sub
    Demcount/RepCount: Total number of posts id'd as Democratic/Republican after coersion
    DemScore/RepScore: Sum of predict_probas for each party. Used as an overall strength metric. 
    """
    return {'subreddit': df_data_current_.iloc[0, :]['subreddit'], 
            'Init_Intensity': len(df_cur_predictions[df_cur_predictions.Party != 0]),
            'DemCount':df_cur_predictions['NewDem'].sum(), 'RepCount': df_cur_predictions['NewDem'].count() - df_cur_predictions['NewDem'].sum(), 
            'DemScore':df_cur_predictions['dem_score'].sum(),'RepScore':df_cur_predictions['rep_score'].sum()}

In [146]:
#Load in previously scraped data
subs_list = ['all10018.csv', 'AskReddit3668.csv', 'askscience72983.csv', 'asoiaf36417.csv', 'aww61461.csv', 'blog1972.csv', 'books78373.csv', 
             'cats105.csv', 'CFB14986.csv', 'Communism91999.csv', 'Democrats18076.csv', 'demsbPosts_9_04_B.csv', 'dogs48732.csv', 'EarthPorn86917.csv', 
             'explainlikeimfive24796.csv', 'funny68372.csv', 'gameofthrones38795.csv', 'gaming7875.csv', 'gifs19936.csv', 'hbbPosts_9_04_A.csv', 'IAmA1082.csv', 
             'Liberal35849.csv', 'liberalPosts_9_04_A.csv', 'Libertarian41804.csv', 'malefashionadvice23143.csv', 'movies84742.csv', 'Music33178.csv', 'news44548.csv', 
             'nfl69766.csv', 'NSFW51148.csv', 'pics99690.csv', 'Republican58200.csv', 'science64623.csv', 'soccer30540.csv', 'television12495.csv', 'the_donald68589.csv', 
             'todayilearned76801.csv', 'videos19793.csv', 'worldnews10804.csv']

list_of_subs = []
for sub in subs_list:
    list_of_subs.append(subreddit_Analysis( pd.read_csv('./Data/'+sub) ))

In [147]:
Check = pd.DataFrame(list_of_subs)

In [148]:
df_clean = Check

In [150]:
df_clean.sort_values(by='subreddit')

,DemCount,DemScore,Init_Intensity,RepCount,RepScore,subreddit
1,319,99.518067,59,182,79.057776,AskReddit
0,324,100.109795,35,301,92.277639,BikiniBottomTwitter
8,169,80.100351,72,333,125.196949,CFB
13,285,71.762272,8,215,65.511076,EarthPorn
19,389,53.139409,1,112,34.756959,HumansBeingBros
20,146,73.611905,111,222,102.642639,IAmA
21,281,218.417835,330,219,167.679737,Liberal
22,277,216.947849,329,223,167.708941,Liberal
23,230,134.706859,162,272,130.895620,Libertarian
26,222,77.350941,19,280,85.442775,Music


In [151]:
#Drop bad scrapes for lack of signficance
df_clean['CombinedCount'] = df_clean['DemCount'] + df_clean['RepCount']
df_clean = df_clean.loc[df_clean['CombinedCount']>150]
df_clean.sort_values(by='subreddit').head()

,DemCount,DemScore,Init_Intensity,RepCount,RepScore,subreddit,CombinedCount
1,319,99.518067,59,182,79.057776,AskReddit,501
0,324,100.109795,35,301,92.277639,BikiniBottomTwitter,625
8,169,80.100351,72,333,125.196949,CFB,502
13,285,71.762272,8,215,65.511076,EarthPorn,500
19,389,53.139409,1,112,34.756959,HumansBeingBros,501


In [152]:
df_clean['PolClass'] =1

In [153]:
#Identify all posts as either Democratic or republican
df_clean['PolClass'].where(df_clean['DemCount']>df_clean['RepCount'], 'Republican', inplace=True)
df_clean['PolClass'].where(df_clean['DemCount']<=df_clean['RepCount'], 'Democrat', inplace=True)
df_clean.reset_index(inplace=True)
df_clean.drop('index', axis=1, inplace=True)

In [154]:
df_clean.sort_values(by='subreddit')

,DemCount,DemScore,Init_Intensity,RepCount,RepScore,subreddit,CombinedCount,PolClass
1,319,99.518067,59,182,79.057776,AskReddit,501,Democrat
0,324,100.109795,35,301,92.277639,BikiniBottomTwitter,625,Democrat
8,169,80.100351,72,333,125.196949,CFB,502,Republican
13,285,71.762272,8,215,65.511076,EarthPorn,500,Democrat
19,389,53.139409,1,112,34.756959,HumansBeingBros,501,Democrat
20,146,73.611905,111,222,102.642639,IAmA,368,Republican
21,281,218.417835,330,219,167.679737,Liberal,500,Democrat
22,277,216.947849,329,223,167.708941,Liberal,500,Democrat
23,230,134.706859,162,272,130.895620,Libertarian,502,Republican
26,222,77.350941,19,280,85.442775,Music,502,Republican


In [155]:
df_clean.drop([11,21], inplace=True)
df_clean.reset_index(drop= True, inplace=True)

In [158]:
df_clean.at[0,'subreddit'] = 'All'

In [159]:
df_clean

,DemCount,DemScore,Init_Intensity,RepCount,RepScore,subreddit,CombinedCount,PolClass
0,324,100.109795,35,301,92.277639,All,625,Democrat
1,319,99.518067,59,182,79.057776,AskReddit,501,Democrat
2,325,143.350884,123,177,104.466674,askscience,502,Democrat
3,268,89.077564,27,234,80.128627,asoiaf,502,Democrat
4,360,56.277141,2,141,40.495514,aww,501,Democrat
5,190,61.225108,39,212,76.495474,blog,402,Republican
6,227,85.584583,52,203,80.243258,books,430,Democrat
7,368,54.398517,3,132,39.270257,cats,500,Democrat
8,169,80.100351,72,333,125.196949,CFB,502,Republican
9,213,115.473149,140,287,136.667035,communism,500,Republican


In [161]:
df_clean.to_csv('./Data/Results_Table.csv', index=False)